# cert-manager
Jetstack's cert-manager is a tool for the creation and management of TLS certificates in Kubernetes.  It supports LetsEncrypt's ACME protocol (Automated Certificate Management Envirionment), as well as several other automated TLS certificate issuers, to automate the process of creating and renewing TLS/HTTPS certificates.  

The ACME protocol provides three methods for proving ownership of a domain: HTTP-01, DNS-01, and TLS-ALPN-01.  HTTP-01 requires that a non-encrypted HTTP: website with special content is published for the domain.  When the Issuer is able to verify the web page, it validates ownership and issues certificates.  DNS-01 publishes a TXT DNS record with a known value to prove ownership.  TLS-ALPN-01 is similar to HTTP-01, but works with only TLS-encrypted HTTPS endpoints.  TLS-ALPN-01 is supported only by a handful of providers currently, and HTTP-01 is limited to only non-wildcard TLS certificates.  Both TLS-ALPN-01 and HTTP-01 require that a publicly accessible webpage be made available, which does not work for purely private k8s clusters.  For these reasons, we recommend the use of DNS-01.

## Prerequisites
- [Learning Jupyter](../getting-started/learning_jupyter.ipynb)
- Install Jupyter bash kernel:
  - Windows... coming soon!
  - [Mac](../getting-started/mac/install_jupyter_bash_kernel.ipynb)
  - [Linux](../getting-started/linux/install_jupyter_bash_kernel.ipynb)
- [Install Kubernetes](../kubernetes/install_k8s_cluster.ipynb)
- Configure DNS
  - Mac: [Install DNSmasq](../dnsmasq/mac/install_dnsmasq.ipynb)
  - Linux: Install DNSmasmq... coming soon!
  - Windows: Install Acrylic... coming soon!
- [Install Monitoring](../monitoring/install_kube-prometheus-stack.ipynb)
- [Install Logging](../logging/install_loki.ipynb)

## Install cert-manager
To install the cert-manager chart, we first must add the Jetstack Helm repository:

In [2]:
helm repo add jetstack https://charts.jetstack.io

"jetstack" already exists with the same configuration, skipping


Next we install cert-manager from the official Helm chart, with a few custom values in our local .yaml file:

In [1]:
helm upgrade --install cert-manager jetstack/cert-manager -n cert-manager --create-namespace -f values.cert-manager.yaml --wait

Release "cert-manager" does not exist. Installing it now.
NAME: cert-manager
LAST DEPLOYED: Mon Mar 28 17:14:56 2022
NAMESPACE: cert-manager
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
cert-manager v1.7.2 has been deployed successfully!

In order to begin issuing certificates, you will need to set up a ClusterIssuer
or Issuer resource (for example, by creating a 'letsencrypt-staging' issuer).

More information on the different types of issuers and how to configure them
can be found in our documentation:

https://cert-manager.io/docs/configuration/

For information on how to configure cert-manager to automatically provision
Certificates for Ingress resources, take a look at the `ingress-shim`
documentation:

https://cert-manager.io/docs/usage/ingress/


: 1

## Setup DNS Provider
Our DNS provider, Cloudflare, supports the ACME standard for issuing TLS certificates, which includes the DNS-01 method to prove domain ownership.  With Cloudflare, we use an API token for our DNS account to create a DNS TXT entry (essentially, a note that has no effects).  Cert-manager sets a special value in a DNS TXT entry, and then checks a public DNS server to ensure that the field has been set.  Once this check is complete, ownership of the domain has been proven and cert-manager is allowed to issue TLS certificates.

To begin this process, we must first install a Cloudflare-issued API token as a secret to Kubernetes, as outlined at https://cert-manager.io/docs/configuration/acme/dns01/cloudflare/ (for other DNS providers, see the [cert-manager documentation](https://cert-manager.io/docs/configuration/acme/dns01/). 

<div class="alert alert-block alert-info">
    <b>Note: </b> 
You must create your own secret here, as per the instructions provided by your DNS provider.  We do NOT include the secret we use, as this secret grants full access to our DNS provider.  Keep the contents of this secret secure!
</div>

In [2]:
kubectl apply -f secret.fake.yaml -n cert-manager

error: the path "secret.yaml" does not exist


: 1

## Setup ClusterIssuer
Next we must setup an Issuer or a ClusterIssuer.  These define the TLS issuer process--in our case, using the DNS-01 specification defined by the ACME protocol.  An Issuer works only for a single k8s Namespace, while a ClusterIssuer works across any namespace.  Here we define a ClusterIssuer that can issue certificates for the DNS Zone "k8s.nephtek.com", which includes all DNS subdomains under, and including, "k8s.nephtek.com".  The ClusterIssuer is configured to use the secret, "cloudflare-api-token-secret", that we just created, for the API token required by Cloudflare.  Finally, the ClusterIssuer includes an email address to notify when there are problems with a certificate, such as if a certificate must be revoked, or a certificate is about to expire and has not been renewed.

In [3]:
kubectl apply -f clusterissuer.nephtek.yaml

clusterissuer.cert-manager.io/nephtek-com-issuer created


: 1

## Create TLS Certificate
Once the ClusterIssuer is Ready, we can request TLS certificates to be issued by the ClusterIssuer.  Since we used the DNS-01 specification, we even can request wildcard certificates.  This certificate works for the base domain, k8s.nephtek.com, and any subdomain immediately under k8s.nephtek.com. 

<div class="alert alert-block alert-info">
    <b>Note: </b> 
You will need to modify the certificate request to a subdomain that you own instead of `nephtek.com`.
</div>

In [4]:
kubectl apply -f certificate.nephtek.yaml

certificate.cert-manager.io/k8s-nephtek-com created


: 1

### Monitor the TLS Certificate Issuing Process
It takes several minutes to complete the TLS issuing process.  While this process is in progress, you can get or describe your certificate to see if it is ready:

In [6]:
kubectl get certificate -n cert-manager

NAME                       READY   SECRET                  AGE
keycloak-k8s-nephtek-com   True    sso-x509-https-secret   36m
k8s-nephtek-com            False   k8s-nephtek-com-tls     20s


If the certificate is not yet ready, there should be an Order associated with it that is in a 'pending' state:

In [7]:
kubectl get order -n cert-manager

NAME                                       STATE     AGE
keycloak-k8s-nephtek-com-5bc4j-997927255   valid     36m
k8s-nephtek-com-flzxc-287383746            pending   25s


Subsequently, there also will be a Challenge for each DNS name in the certificate request.  Since we are requesting a certificate that is valid for both "\*.k8s.nephtek.com" and "k8s.nephtek.com", we have two challenges for this certificate:

In [8]:
kubectl get challenge -n cert-manager

NAME                                         STATE     DOMAIN            AGE
k8s-nephtek-com-flzxc-287383746-4001887644             k8s.nephtek.com   30s
k8s-nephtek-com-flzxc-287383746-2388122080   pending   k8s.nephtek.com   30s


Once the process is complete, usually in about 2-5 minutes, the Challenge will disappear:

In [9]:
kubectl get challenge -n cert-manager

No resources found in keycloak namespace.


...and the order will move from 'pending' to 'valid':

In [ ]:
kubectl get order -n cert-manager

NAME                                       STATE   AGE
keycloak-k8s-nephtek-com-5bc4j-997927255   valid   41m
k8s-nephtek-com-flzxc-287383746            valid   4m27s


...and the Certificate readiness will move from 'False' to 'True':

In [12]:
kubectl get cert -n cert-manager

NAME                       READY   SECRET                  AGE
keycloak-k8s-nephtek-com   True    sso-x509-https-secret   41m
k8s-nephtek-com            True    k8s-nephtek-com-tls     4m41s


When this is complete, the contents of the secret will contain a valid TLS certificate:

In [13]:
kubectl describe secret k8s-nephtek-com-tls -n cert-manager

Name:         k8s-nephtek-com-tls
Namespace:    keycloak
Labels:       <none>
Annotations:  cert-manager.io/alt-names: *.k8s.nephtek.com,k8s.nephtek.com
              cert-manager.io/certificate-name: k8s-nephtek-com
              cert-manager.io/common-name: *.k8s.nephtek.com
              cert-manager.io/ip-sans: 
              cert-manager.io/issuer-group: cert-manager.io
              cert-manager.io/issuer-kind: ClusterIssuer
              cert-manager.io/issuer-name: nephtek-com-issuer
              cert-manager.io/uri-sans: 

Type:  kubernetes.io/tls

Data
====
tls.crt:  5623 bytes
tls.key:  1675 bytes


### Finished

At this point you should have cert-manager configured, a ClusterIssuer that is capable of issuing valid TLS certificates, and a wildcard certificate for your base URL.

### Next Steps
- [Install Ingress Controller](../ingress/install_ingress-nginx.ipynb)
